In [ ]:
Xgboost(Extreme Gradient Boosting) : 
similar to boosting framework but more efficinet. It has both linear model solver and Decision tree learning algo.

--> what makes it fast its capacity to do parallel computation on a single machine.
This makes xgboost atleast 10 times faster than existing gradient boosting implementations. It supports various 
objective functions, including regression, classification and ranking.
It also has additional feature for doing cross validation and finding important var.

Weak classifier : an algorithm slightly better than random guessing.
    
Boosting tech. : try to reduce the variance by averaging many different trees(where each one solving the same problem).

Every algorithm can be used as a base for boosting technique, but trees have some nice properties that makes them 
more suitable for guessing the problem.

### Pro's

In [ ]:
-computational scalability 
-handle missing values automatically
-robust to outliers
-not require feature scaling(automatically hapens in algo.)
-deal with irrelevant inputs.
-interpretable(if small)
-can handle mixed predictors(quantative and qualitative) quantative- regression kind of problems(find house prices,stock price) 
    qualitative-classifier predict 2type of values 0,1 or True-False

### Con's

In [ ]:
-can't extract linear combination of features
-small predictive power(high variance)

##### In dataset we have information of Bank , in Exited column we have 1 for user left the Bank and 0 for not left (Or Retained). CustomerId, Surname column are not reqd. and all are independent features and Exited column - dependent feature. 
##### dataset downloaded from  Kaggle : https://www.kaggle.com/aakash50897/churn-modellingcsv

In [15]:
import pandas as pd                                               # conda install -c anaconda py-xgboost 
import numpy as np
import matplotlib.pyplot as plt

In [16]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [17]:
X = dataset.iloc[:,3:13].values   # indepedent features
y = dataset.iloc[:,13].values    # depedent feature 

In [18]:
# Now we encode categorical data (using label encoding: 0,1 and OneHotEncoder) In dataset we have Geography, Gender column

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

labelencoder_X_1 = LabelEncoder()
X[:,1] = labelencoder_X_1.fit_transform(X[:,1])       # for 1st column Geography

labelencoder_X_2 = LabelEncoder()
X[:,2] = labelencoder_X_2.fit_transform(X[:,2])       # for 2nd column Gender

In [19]:
X

array([[619, 0, 0, ..., 1, 1, 101348.88],
       [608, 2, 0, ..., 0, 1, 112542.58],
       [502, 0, 0, ..., 1, 0, 113931.57],
       ...,
       [709, 0, 0, ..., 0, 1, 42085.58],
       [772, 1, 1, ..., 1, 0, 92888.52],
       [792, 0, 0, ..., 1, 0, 38190.78]], dtype=object)

In [21]:
onehotencoder = OneHotEncoder(categorical_features=[1])

X = onehotencoder.fit_transform(X).toarray()   # converting into array
X = X[:,1:]      # deleting 1st element - dummy var.trap

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:392: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [22]:
X

array([[0.0000000e+00, 0.0000000e+00, 6.1900000e+02, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [1.0000000e+00, 0.0000000e+00, 6.0800000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [0.0000000e+00, 0.0000000e+00, 5.0200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [0.0000000e+00, 0.0000000e+00, 7.0900000e+02, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [0.0000000e+00, 1.0000000e+00, 7.7200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [0.0000000e+00, 0.0000000e+00, 7.9200000e+02, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [23]:
from sklearn.model_selection import train_test_split      # train test split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

#### fit Xgboost model

In [24]:
from xgboost import XGBClassifier                  # creating model

classifier = XGBClassifier(max_depth=5)
classifier.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [25]:
y_pred = classifier.predict(X_test)                              # predict

# we can also apply K-Fold cross validation , change training dataset

In [39]:
# Now we compare y_pred, y_test

from sklearn.metrics import confusion_matrix, accuracy_score

cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)

In [40]:
print('Confusion matrix : \n', cm)
print(f'\n Accuracy is : {acc:0.2%}')

Confusion matrix : 
 [[1522   73]
 [ 186  219]]
Accuracy is : 87.05%
